In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc

import plotly
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [2]:
from rpy2 import robjects
from rpy2.robjects.packages import importr
ocp = importr('ocp')
dplyr = importr('dplyr')
r = robjects.r
r.source('R-script.R')

value,[RTYPES.CLOSXP]
visible,[RTYPES.LGLSXP]


In [3]:
app = dash.Dash()

In [4]:
allData = pd.read_csv("goodListShort.csv") 
filteredADMISSION = pd.read_csv("filteredADMISSION.csv")
filteredINPUTEVENTS = pd.read_csv("inputFilter.csv")
D_ITEMS = pd.read_csv("D_ITEMS.csv")
P_F = pd.read_csv("PaO2_FaO2.csv")
score = pd.read_csv("score.csv")
temp_re = pd.read_csv("01temp_re.csv")
abp_res = pd.read_csv("02abp_res.csv")
Bilirubin_res = pd.read_csv("03Bilirubin_res.csv")
Creatinine_res = pd.read_csv("04Creatinine_res.csv")
GCS_res = pd.read_csv("05GCS_res.csv")
MAP_res = pd.read_csv("06MAP_res.csv")
Platelet_res = pd.read_csv("07Platelet_res.csv")
WBC_res = pd.read_csv("08WBC_res.csv")
PaO2_res = pd.read_csv("09PaO2_res.csv")
Urine_res = pd.read_csv("10Urine_res.csv")
INR_res = pd.read_csv("11INR_res.csv")

In [5]:
df = allData.sort_values(by = 'CHARTTIME')
dfINPUTEVENTS = filteredINPUTEVENTS.sort_values(by = 'Start.Time')

In [6]:
dfINPUTEVENTS.columns.values.tolist() 

['Row.ID',
 'Subject.ID',
 'Hospital.Admin.ID',
 'ICU.STAY.ID',
 'Start.Time',
 'End.Time',
 'Item.ID',
 'Amount',
 'AmountUOM',
 'Rate',
 'RATEUOM',
 'STORETIME',
 'CGID',
 'ORDERID',
 'LINKORDERID',
 'ORDERCATEGORYNAME',
 'SECONDARYORDERCATEGORYNAME',
 'ORDERCOMPONENTTYPEDESCRIPTION',
 'ORDERCATEGORYDESCRIPTION',
 'PATIENTWEIGHT',
 'TOTALAMOUNT',
 'Total.Amount.UOM',
 'Is.Open.Bag',
 'Continue.in.Next.Department',
 'Cancel.Reason',
 'Status.Description',
 'Comments.Edited.By',
 'Comments.Canceled.By',
 'Comments.Date',
 'Original.Amount',
 'Original.Rate',
 'CATEGORY',
 'PARAM_TYPE']

In [7]:
inputData = pd.DataFrame(columns = ["Hospital Admin ID", "num", "Item", "Category",
                                    "Parameter Type",
                                    "Amount", "Amount UOM", "Rate", "Rate UoM",
                                   "Start Time", "End Time", 
                                    "Order Component Type", "Order Category",
                                   "Patient Weight", "Dose Amount", "Dose UoM",
                                   "Status"])

In [8]:
inputData['Hospital Admin ID'] = dfINPUTEVENTS['Hospital.Admin.ID']
inputData['Item'] = dfINPUTEVENTS['Item.ID']
inputData['Category'] = dfINPUTEVENTS['CATEGORY']
inputData['Parameter Type'] = dfINPUTEVENTS['PARAM_TYPE']
inputData['Amount'] = dfINPUTEVENTS['Amount']
inputData['Amount UOM'] = dfINPUTEVENTS['AmountUOM']
inputData['Rate'] = dfINPUTEVENTS['Rate']
inputData['Rate UoM'] = dfINPUTEVENTS['RATEUOM']
inputData['Start Time'] = dfINPUTEVENTS['Start.Time']
inputData['End Time'] = dfINPUTEVENTS['End.Time']
inputData['Order Component Type'] = dfINPUTEVENTS['ORDERCOMPONENTTYPEDESCRIPTION']
inputData['Order Category'] = dfINPUTEVENTS['ORDERCATEGORYNAME']
inputData['Patient Weight'] = dfINPUTEVENTS['PATIENTWEIGHT']
inputData['Dose Amount'] = dfINPUTEVENTS['TOTALAMOUNT']
inputData['Dose UoM'] = dfINPUTEVENTS['Total.Amount.UOM']
inputData['Status'] = dfINPUTEVENTS['Status.Description']

In [9]:
HADMID = pd.DataFrame(df['HADM_ID'].unique())

In [10]:
HADMID = HADMID.sort_values(by = 0)

In [11]:
df.head

<bound method NDFrame.head of               X  HADM_ID            CHARTTIME  VALUENUM  \
30028   4101101   122354  2102-02-16 18:17:00       1.0   
30029    210895   122354  2102-02-16 18:19:00      22.0   
30030    592627   122354  2102-02-16 18:19:00     118.0   
30031    210896   122354  2102-02-16 18:20:00      17.0   
30032    592628   122354  2102-02-16 18:20:00     113.0   
...         ...      ...                  ...       ...   
137146   453884   191708  2201-04-21 12:36:00      15.0   
137148   568949   191708  2201-04-21 13:00:00      77.0   
137147   410988   191708  2201-04-21 13:00:00      25.0   
137149  2098211   191708  2201-04-22 04:38:00     202.0   
137150  2098311   191708  2201-04-23 04:03:00     243.0   

                              LABEL          VALUEUOM  
30028   Fraction of Inspired Oxygen  Unitless (Ratio)  
30029              Respiratory Rate               bpm  
30030                    Heart Rate               BPM  
30031              Respiratory Rate  

In [12]:
temperature = df[df['LABEL'] == 'Temperature']
respiratory = df[df['LABEL'] == 'Respiratory Rate']
heart = df[df['LABEL'] == 'Heart Rate']
WBC = df[df['LABEL'] == 'White Blood Cell Count']
GCS = df[df['LABEL'] == 'Glasgow Coma Scale']
ABP = df[df['LABEL'] == 'Systolic Arterial Blood Pressure']
PaO2 = P_F
Platelet = df[df['LABEL'] == 'Platelet Count']
Bilirubin = df[df['LABEL'] == 'Bilirubin Level']
MAP= df[df['LABEL'] == 'Mean Arterial Pressure']
Creatinine = df[df['LABEL'] == 'Creatinine']

In [13]:
scoredf = score.sort_values(by = 'VALUENUM')

heartTrace = scoredf[(scoredf['VALUENUM'] > 20)|(scoredf['HeartRate'] > 90)]
temp_reTrace = temp_re[(temp_re['RespiratoryRate'] > 20)|(temp_re['VALUENUM'] > 100.4)|(temp_re['VALUENUM'] < 96.8)]
abp_resTrace = abp_res[(abp_res['RespiratoryRate'] > 20)|(abp_res['VALUENUM'] <= 100)]
Bilirubin_resTrace = Bilirubin_res[(Bilirubin_res['RespiratoryRate'] > 20)|(Bilirubin_res['VALUENUM'] >= 2)]
Creatinine_resTrace = Creatinine_res[(Creatinine_res['RespiratoryRate'] > 20)|(Creatinine_res['VALUENUM'] >= 2)]
GCS_resTrace = GCS_res[(GCS_res['RespiratoryRate'] > 20)|(GCS_res['VALUENUM'] <= 12)]
MAP_resTrace = MAP_res[(MAP_res['RespiratoryRate'] > 20)|(MAP_res['VALUENUM'] < 65)]
Platelet_resTrace = Platelet_res[(Platelet_res['RespiratoryRate'] > 20)|(Platelet_res['VALUENUM'] < 100)]
WBC_resTrace = WBC_res[(WBC_res['RespiratoryRate'] > 20)|(WBC_res['VALUENUM'] > 12) |(WBC_res['VALUENUM'] < 4)]
PaO2_resTrace = PaO2_res[(PaO2_res['RespiratoryRate'] > 20)|(PaO2_res['VALUENUM'] <= 300)]
Urine_resTrace = Urine_res[(Urine_res['RespiratoryRate'] > 20)|(Urine_res['VALUE'] <= 0.5)]
INR_resTrace = INR_res[(INR_res['RespiratoryRate'] > 20)|(INR_res['VALUENUM'] > 1.5)]

In [14]:
app.layout = html.Div([
    html.Center(html.H1 ("Sepsis")),
    html.Div([
        html.P ("HADM_ID"),
        html.Div([
            html.Div([
                dcc.Dropdown(
                id='HADM_ID',
                options=[{'label': i, 'value': i} for i in HADMID[0]],
                value= 100784
                ),
                html.P ("Method"),
                dcc.Dropdown(
                    id='sort',
                    options=[
                        {'label': 'Sepsis-2', 'value': 0},
                        {'label': 'Sepsis-3', 'value': 1},
                        {'label': 'Scatter plot', 'value': 3}],
                    value= 0,
                ),
            ], style={'width': '48%', 'display': 'inline-block', 'height': 200}),
            html.Div(id='info',
                     style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
        ]),

        html.Div([
                dcc.Graph(id='input_data')
            ]),
        html.Div(id='my-div')

])])

@app.callback(
    dash.dependencies.Output(component_id='my-div', component_property='children'),
    [dash.dependencies.Input('HADM_ID', 'value'),
     dash.dependencies.Input('sort', 'value')])
def update_div(HADM,sort):
    if (sort == 0):
        SIRS_fig = go.Figure(
            data=[go.Scatter(x=score[score['HADM_ID'] == HADM]['CHARTTIME'],
                                          y=score[score['HADM_ID'] == HADM]['SIRS'],
                            line=dict(shape='hvh'),
                            mode='lines')])
        SIRS_fig.update_layout(
            title={
                'text': "Sepsis",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            yaxis = dict(
                tickmode = "array",
                tickvals = [0,1,2,3],
                ticktext = ['non-Sepsis','Sepsis','Severe Sepsis','Septic Shock']
            )
        )
        
        
        fig_RespiratoryRate = go.Figure(
            data=[go.Scatter(x=respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'],
                             y=respiratory[respiratory['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_RespiratoryRate.update_layout(
            title={
                'text': "Respiratory Rate",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Respiratory Rate"
        )
        R_RespiratoryRate = robjects.FloatVector(respiratory[respiratory['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_RespiratoryRate = list(r.ChangePoint(R_RespiratoryRate,0.5))
        for changes in change_RespiratoryRate:
            if((int(changes)-1)!=0):
                fig_RespiratoryRate.add_trace(go.Scatter(
                    x=[respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[-5,40],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
            
        
        
        fig_HeartRate = go.Figure(
            data=[go.Scatter(x=heart[heart['HADM_ID'] == HADM]['CHARTTIME'],
                             y=heart[heart['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_HeartRate.update_layout(
            title={
                'text': "Heart Rate",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Heart Rate"
        )
        R_HeartRate = robjects.FloatVector(heart[heart['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_HeartRate = list(r.ChangePoint(R_HeartRate,0.5))
        for changes in change_HeartRate:
            if((int(changes)-1)!=0):
                fig_HeartRate.add_trace(go.Scatter(
                    x=[heart[heart['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       heart[heart['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[40,110],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}',
                ))
            
            
        fig_Temperature = go.Figure(
            data=[go.Scatter(x=temperature[temperature['HADM_ID'] == HADM]['CHARTTIME'],
                             y=temperature[temperature['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_Temperature.update_layout(
            title={
                'text': "Temperature",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Temperature"
        )
        R_Temperature = robjects.FloatVector(temperature[temperature['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_Temperature = list(r.ChangePoint(R_Temperature,0.5))
        for changes in change_Temperature:
            if((int(changes)-1)!=0):
                fig_Temperature.add_trace(go.Scatter(
                    x=[temperature[temperature['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       temperature[temperature['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[90,110],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}',
                ))
            
        fig_WBC = go.Figure(
            data=[go.Scatter(x=WBC[WBC['HADM_ID'] == HADM]['CHARTTIME'],
                             y=WBC[WBC['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_WBC.update_layout(
            title={
                'text': "WBC",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="WBC"
        )
        R_WBC = robjects.FloatVector(WBC[WBC['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_WBC = list(r.ChangePoint(R_WBC,0.5))
        for changes in change_WBC:
            if((int(changes)-1)!=0):
                fig_WBC.add_trace(go.Scatter(
                    x=[WBC[WBC['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       WBC[WBC['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[0,30],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}',
                ))
        
        return [
            dcc.Graph(
                figure=SIRS_fig),
            html.Center(html.H2 ("SIRS")),
            dcc.Graph(
                figure = fig_RespiratoryRate),
            dcc.Graph(
                figure = fig_HeartRate),
            dcc.Graph(
                figure = fig_Temperature),
            dcc.Graph(
                figure = fig_WBC)]
    if (sort == 1):
        SOFA_fig = go.Figure(
            data=[go.Scatter(x=score[score['HADM_ID'] == HADM]['CHARTTIME'],
                                          y=score[score['HADM_ID'] == HADM]['score'],
                            line=dict(shape='hvh'),
                            mode='lines')])
        SOFA_fig.update_layout(
            title={
                'text': "Sepsis",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            yaxis = dict(
                tickmode = "array",
                tickvals = [0,1,2],
                ticktext = ['non-Sepsis', 'Sepsis','Sepsis shock']
            )
        )
        
        fig_RespiratoryRate = go.Figure(
            data=[go.Scatter(x=respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'],
                             y=respiratory[respiratory['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_RespiratoryRate.update_layout(
            title={
                'text': "Respiratory Rate",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Respiratory Rate"
        )
        R_RespiratoryRate = robjects.FloatVector(respiratory[respiratory['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_RespiratoryRate = list(r.ChangePoint(R_RespiratoryRate,0.5))
        for changes in change_RespiratoryRate:
            if((int(changes)-1)!=0):
                fig_RespiratoryRate.add_trace(go.Scatter(
                    x=[respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       respiratory[respiratory['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[-5,40],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
                
        fig_GCS = go.Figure(
            data=[go.Scatter(x=GCS[GCS['HADM_ID'] == HADM]['CHARTTIME'],
                             y=GCS[GCS['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_GCS.update_layout(
            title={
                'text': "GCS",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="GCS"
        )
        R_GCS = robjects.FloatVector(GCS[GCS['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_GCS = list(r.ChangePoint(R_GCS,0.5))
        for changes in change_GCS:
            if((int(changes)-1)!=0):
                fig_GCS.add_trace(go.Scatter(
                    x=[GCS[GCS['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       GCS[GCS['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[-1,16],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
                
        fig_ABP = go.Figure(
            data=[go.Scatter(x=ABP[ABP['HADM_ID'] == HADM]['CHARTTIME'],
                             y=ABP[ABP['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_ABP.update_layout(
            title={
                'text': "ABP [Systolic]",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="ABP [Systolic]"
        )
        R_ABP = robjects.FloatVector(ABP[ABP['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_ABP = list(r.ChangePoint(R_ABP,0.5))
        for changes in change_ABP:
            if((int(changes)-1)!=0):
                fig_ABP.add_trace(go.Scatter(
                    x=[ABP[ABP['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       ABP[ABP['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[40,205],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
                
        fig_PaO2 = go.Figure(
            data=[go.Scatter(x=PaO2[PaO2['HADM_ID'] == HADM]['CHARTTIME'],
                             y=PaO2[PaO2['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_PaO2.update_layout(
            title={
                'text': "PaO2/FIO2",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="PaO2/FIO2"
        )
        R_PaO2 = robjects.FloatVector(PaO2[PaO2['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_PaO2 = list(r.ChangePoint(R_PaO2,0.5))
        for changes in change_PaO2:
            if((int(changes)-1)!=0):
                fig_PaO2.add_trace(go.Scatter(
                    x=[PaO2[PaO2['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       PaO2[PaO2['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[0,600],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
        
        fig_Platelet = go.Figure(
            data=[go.Scatter(x=Platelet[Platelet['HADM_ID'] == HADM]['CHARTTIME'],
                             y=Platelet[Platelet['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_Platelet.update_layout(
            title={
                'text': "Platelet",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Platelet"
        )
        R_Platelet = robjects.FloatVector(Platelet[Platelet['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_Platelet = list(r.ChangePoint(R_Platelet,0.5))
        for changes in change_Platelet:
            if((int(changes)-1)!=0):
                fig_Platelet.add_trace(go.Scatter(
                    x=[Platelet[Platelet['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       Platelet[Platelet['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[40,350],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
        
        fig_Bilirubin = go.Figure(
            data=[go.Scatter(x=Bilirubin[Bilirubin['HADM_ID'] == HADM]['CHARTTIME'],
                             y=Bilirubin[Bilirubin['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_Bilirubin.update_layout(
            title={
                'text': "Bilirubin",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Bilirubin"
        )
        R_Bilirubin = robjects.FloatVector(Bilirubin[Bilirubin['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_Bilirubin = list(r.ChangePoint(R_Bilirubin,0.5))
        for changes in change_Bilirubin:
            if((int(changes)-1)!=0):
                fig_Bilirubin.add_trace(go.Scatter(
                    x=[Bilirubin[Bilirubin['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       Bilirubin[Bilirubin['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[0,55],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
        
        fig_MAP = go.Figure(
            data=[go.Scatter(x=MAP[MAP['HADM_ID'] == HADM]['CHARTTIME'],
                             y=MAP[MAP['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_MAP.update_layout(
            title={
                'text': "Mean Arterial Pressure",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Mean Arterial Pressure"
        )
        R_MAP = robjects.FloatVector(MAP[MAP['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_MAP = list(r.ChangePoint(R_MAP,0.5))
        for changes in change_MAP:
            if((int(changes)-1)!=0):
                fig_MAP.add_trace(go.Scatter(
                    x=[MAP[MAP['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       MAP[MAP['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[40,350],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
        
        fig_Creatinine = go.Figure(
            data=[go.Scatter(x=Creatinine[Creatinine['HADM_ID'] == HADM]['CHARTTIME'],
                             y=Creatinine[Creatinine['HADM_ID'] == HADM]['VALUENUM'],
                             showlegend=False,
                             mode='markers+lines')])
        fig_Creatinine.update_layout(
            title={
                'text': "Creatinine",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Time",
            yaxis_title="Creatinine"
        )
        R_Creatinine = robjects.FloatVector(Creatinine[Creatinine['HADM_ID'] == HADM]['VALUENUM'].tolist())
        change_Creatinine = list(r.ChangePoint(R_Creatinine,0.5))
        for changes in change_Creatinine:
            if((int(changes)-1)!=0):
                fig_Creatinine.add_trace(go.Scatter(
                    x=[Creatinine[Creatinine['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)], 
                       Creatinine[Creatinine['HADM_ID'] == HADM]['CHARTTIME'].tolist()[(int(changes)-1)]],
                    y=[0,10],
                    mode='lines',
                    line=dict(color='rgb(182, 44, 45)'),
                    showlegend=False,
                    name = 'Change Point',
                    hovertemplate = '%{x}'
                ))
        
        return [
            dcc.Graph(figure=SOFA_fig),
            html.Center(html.H2 ("qSOFA")),
            dcc.Graph(figure = fig_RespiratoryRate),
            dcc.Graph(figure = fig_GCS),
            dcc.Graph(figure = fig_ABP),
            html.Center(html.H2 ("SOFA")),
            dcc.Graph(figure = fig_PaO2),
            dcc.Graph(figure = fig_Platelet),
            dcc.Graph(figure = fig_Bilirubin),
            dcc.Graph(figure = fig_MAP),
            dcc.Graph(figure = fig_Creatinine)]
    if (sort == 3):
        HeartRate_fig = go.Figure(
            data=[go.Scatter(x=scoredf[scoredf['HADM_ID'] == HADM]['VALUENUM'],
                             y=scoredf[scoredf['HADM_ID'] == HADM]['HeartRate'],
                             name="All data",
                            mode='markers')])
        HeartRate_fig.update_layout(
            title={
                'text': "Respiratory Rate & Heart Rate",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Heart Rate"
        )
        HeartRate_fig.add_trace(go.Scatter(
            x=heartTrace[heartTrace['HADM_ID'] == HADM]['VALUENUM'],
            y=heartTrace[heartTrace['HADM_ID'] == HADM]['HeartRate'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        
        Temperature_fig1 = go.Figure(
            data=[go.Scatter(x=temp_re[temp_re['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=temp_re[temp_re['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        Temperature_fig1.update_layout(
            title={
                'text': "Respiratory Rate & Temperature",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Temperature")
        Temperature_fig1.add_trace(go.Scatter(
            x=temp_reTrace[temp_reTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=temp_reTrace[temp_reTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        WBC_fig = go.Figure(
            data=[go.Scatter(x=WBC_res[WBC_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=WBC_res[WBC_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        WBC_fig.update_layout(
            title={
                'text': "Respiratory Rate & White Blood Cell",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="White Blood Cell")
        WBC_fig.add_trace(go.Scatter(
            x=WBC_resTrace[WBC_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=WBC_resTrace[WBC_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        GCS_fig = go.Figure(
            data=[go.Scatter(x=GCS_res[GCS_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=GCS_res[GCS_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        GCS_fig.update_layout(
            title={
                'text': "Respiratory Rate & Glasgow Coma Scale",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Glasgow Coma Scale")
        GCS_fig.add_trace(go.Scatter(
            x=GCS_resTrace[GCS_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=GCS_resTrace[GCS_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        ABP_fig = go.Figure(
            data=[go.Scatter(x=abp_res[abp_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=abp_res[abp_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        ABP_fig.update_layout(
            title={
                'text': "Respiratory Rate & Systolic Arterial Blood Pressure",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Systolic Arterial Blood Pressure")
        ABP_fig.add_trace(go.Scatter(
            x=abp_resTrace[abp_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=abp_resTrace[abp_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        PaO2_fig = go.Figure(
            data=[go.Scatter(x=PaO2_res[PaO2_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=PaO2_res[PaO2_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        PaO2_fig.update_layout(
            title={
                'text': "Respiratory Rate & PaO2/FIO2",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="PaO2/FIO2")
        PaO2_fig.add_trace(go.Scatter(
            x=PaO2_resTrace[PaO2_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=PaO2_resTrace[PaO2_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        Platelet_fig = go.Figure(
            data=[go.Scatter(x=Platelet_res[Platelet_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=Platelet_res[Platelet_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        Platelet_fig.update_layout(
            title={
                'text': "Respiratory Rate & Platelets",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Platelets")
        Platelet_fig.add_trace(go.Scatter(
            x=Platelet_resTrace[Platelet_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=Platelet_resTrace[Platelet_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        Bilirubin_fig = go.Figure(
            data=[go.Scatter(x=Bilirubin_res[Bilirubin_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=Bilirubin_res[Bilirubin_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        Bilirubin_fig.update_layout(
            title={
                'text': "Respiratory Rate & Bilirubin",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Bilirubin")
        Bilirubin_fig.add_trace(go.Scatter(
            x=Bilirubin_resTrace[Bilirubin_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=Bilirubin_resTrace[Bilirubin_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        MAP_fig = go.Figure(
            data=[go.Scatter(x=MAP_res[MAP_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=MAP_res[MAP_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        MAP_fig.update_layout(
            title={
                'text': "Respiratory Rate & Mean Arterial Pressure",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Mean Arterial Pressure")
        MAP_fig.add_trace(go.Scatter(
            x=MAP_resTrace[MAP_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=MAP_resTrace[MAP_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        Creatinine_fig = go.Figure(
            data=[go.Scatter(x=Creatinine_res[Creatinine_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=Creatinine_res[Creatinine_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        Creatinine_fig.update_layout(
            title={
                'text': "Respiratory Rate & Creatinine",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Creatinine")
        Creatinine_fig.add_trace(go.Scatter(
            x=Creatinine_resTrace[Creatinine_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=Creatinine_resTrace[Creatinine_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        Urine_fig = go.Figure(
            data=[go.Scatter(x=Urine_res[Urine_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=Urine_res[Urine_res['HADM_ID'] == HADM]['VALUE'],
                             name="All data",
                            mode='markers')])
        Urine_fig.update_layout(
            title={
                'text': "Respiratory Rate & Urine",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="Urine")
        Urine_fig.add_trace(go.Scatter(
            x=Urine_resTrace[Urine_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=Urine_resTrace[Urine_resTrace['HADM_ID'] == HADM]['VALUE'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        INR_fig = go.Figure(
            data=[go.Scatter(x=INR_res[INR_res['HADM_ID'] == HADM]['RespiratoryRate'],
                                          y=INR_res[INR_res['HADM_ID'] == HADM]['VALUENUM'],
                             name="All data",
                            mode='markers')])
        INR_fig.update_layout(
            title={
                'text': "Respiratory Rate & INR",
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
            xaxis_title="Respiratory Rate",
            yaxis_title="INR")
        INR_fig.add_trace(go.Scatter(
            x=INR_resTrace[INR_resTrace['HADM_ID'] == HADM]['RespiratoryRate'],
            y=INR_resTrace[INR_resTrace['HADM_ID'] == HADM]['VALUENUM'],
            mode='markers',
            name="Inside Sepsis Risk Zone"       # this sets its legend entry
        ))
        
        return [html.Div([
            html.Div([
                dcc.Graph(
            figure=HeartRate_fig),
                dcc.Graph(
            figure=Temperature_fig1),
                dcc.Graph(
            figure=WBC_fig),
                dcc.Graph(
            figure=GCS_fig),
                dcc.Graph(
            figure=ABP_fig),
                dcc.Graph(
            figure=Urine_fig)], 
                     style={'width': '48%', 'display': 'inline-block'}),
            html.Div([
                dcc.Graph(
            figure=PaO2_fig),
                dcc.Graph(
            figure=Platelet_fig),
                dcc.Graph(
            figure=Bilirubin_fig),
                dcc.Graph(
            figure=MAP_fig),
                dcc.Graph(
            figure=Creatinine_fig),
                dcc.Graph(
            figure=INR_fig)],
                     style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
        ])
            
            ]
    
    
@app.callback(
    dash.dependencies.Output(component_id='info', component_property='children'),
    [dash.dependencies.Input('HADM_ID', 'value')])
def update_output_div(input_value):
    if (len(filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value])!=0):
        input_value1 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][3]
        input_value2 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][6]
        input_value3 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][7]
        input_value4 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][10]
        input_value5 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][11]
        input_value6 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][12]
        input_value7 = filteredADMISSION[filteredADMISSION['Hospital Admin ID'] == input_value].values[0][13]
        return [html.Div([
            html.Div([html.P('Admit Time: {}'.format(input_value1)),
                      html.P('Admission Type: {}'.format(input_value2)),
                      html.P('Admission Location: {}'.format(input_value3)),
                      html.P('Language: {}'.format(input_value4))],
                              style={'width': '45%', 'display': 'inline-block'}),
            html.Div([html.P('Religion: {}'.format(input_value5)),
                      html.P('Marital Status: {}'.format(input_value6)),
                      html.P('Ethnicity: {}'.format(input_value7))],
                              style={'width': '45%', 'float': 'right', 'display': 'inline-block'})]),

               ]
    return ''

@app.callback(
    dash.dependencies.Output(component_id='input_data', component_property='figure'),
    [dash.dependencies.Input('HADM_ID', 'value')])
def update_div(HADM):
    dff = inputData[inputData['Hospital Admin ID'] == HADM]
    dff = dff.sort_values(by = 'Start Time')
    dff['num'] = range(1,len(dff)+1)
    dff = dff.fillna('No data')
    fig = px.scatter(
            dff,
            x='Start Time',
            y='num',
            hover_data={
                        'num':False,
                        'Item':True,
                        'Category':True,
                        'Parameter Type':True,
                        'Amount':True,
                        'Amount UOM':True,
                        'Rate':True,
                        'Rate UoM':True,
                        'Start Time':True,
                        'End Time':True,
                        'Order Component Type':True,
                        'Order Category':True,
                        'Patient Weight':True,
                        'Dose Amount':True,
                        'Dose UoM':True,
                        'Status':True
            }
    )
    fig.update_layout(yaxis_title="Input Sequence",
                      title={
                          'text': "Input Event",
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'})
    return fig

In [15]:
if __name__ == "__main__":
    app.run_server (port=2570,debug=True,use_reloader=False)

Running on http://127.0.0.1:2570/
Debugger PIN: 903-349-558
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
